# Feature Extraction for Shallow Learners
## TSFresh Feature Extraction
see: https://docs.google.com/presentation/d/1QQYKjV47gLrgohGNP-5UQj67KMwFzrrGEGJyi-y2Ox0/edit#slide=id.g6c263ea923_0_896 slides 51-58

In [30]:
import h5py
import datasets
from importlib import reload
reload(datasets)

<module 'datasets' from '/home/mohammed/eeg-miblab/datasets.py'>

In [2]:

testFile = h5py.File("/datadrive/TUH_EEG/MUPS/data/cross_sub_TUH_EEG_denoised/cross_subject_data_test.hdf5")

/home/mohammed/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  """Entry point for launching an IPython kernel.
/home/mohammed/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:2: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  


In [7]:
train_x = train_edss[0]

In [9]:
from tsfresh import extract_features
def extract(eeg):
    eeg = pd.DataFrame(eeg)
    return extract_features(eeg.unstack().reset_index()[["level_0", 0]], column_id="level_0")

In [11]:
from joblib import Parallel, delayed
import pandas as pd
from tqdm import tqdm
all_dfs = Parallel(n_jobs=12)(delayed(extract)(eeg) for eeg in tqdm(train_x))


  2%|▏         | 1140/46446 [2:35:25<90:44:32,  7.21s/it] 

KeyboardInterrupt: 

## simpler feature extraction

In [52]:
import numpy as np
import constants
import tsfresh.feature_extraction.feature_calculators as tsf
import pickle as pkl
reload(datasets)
freq_bins=[0,3.5,7.5,14,20,25,40]
def extract_train_features():
    with h5py.File("/datadrive/TUH_EEG/MUPS/data/cross_sub_TUH_EEG_denoised/cross_subject_data_train_val.hdf5") as trainValFile:
        train_edss = (trainValFile['train_x'][()], trainValFile['train_y'][()])
        trainCoherData = datasets.CoherenceTransformer(list(zip(train_edss[0], train_edss[1])), is_pandas=False)
        trainSHED = datasets.SimpleHandEngineeredDataset(list(zip(train_edss[0], train_edss[1])), n_process=12, is_pandas_data=False, features=[tsf.abs_energy, tsf.kurtosis, tsf.sample_entropy, lambda x: tsf.number_cwt_peaks(x, int(constants.COMMON_FREQ/25))], f_names=["abs_energy", "kurtosis", "entropy", "num_peaks"], vectorize="full")[:]
        train_edss = datasets.Flattener(datasets.EdfFFTDatasetTransformer(list(zip(train_edss[0], train_edss[1])), freq_bins=freq_bins, is_pandas_data=False), n_process=12)[:]
        pkl.dump(train_edss, open("extracted_train_features_v2.pkl", "wb"))
extract_train_features()
def extract_val_features():
    with h5py.File("/datadrive/TUH_EEG/MUPS/data/cross_sub_TUH_EEG_denoised/cross_subject_data_train_val.hdf5") as trainValFile:
        train_edss = (trainValFile['val_x'][()], trainValFile['val_y'][()])
        trainCoherData = datasets.CoherenceTransformer(list(zip(train_edss[0], train_edss[1])), is_pandas=False)
        trainSHED = datasets.SimpleHandEngineeredDataset(list(zip(train_edss[0], train_edss[1])), n_process=12, is_pandas_data=False, features=[tsf.abs_energy, tsf.kurtosis, tsf.sample_entropy, lambda x: tsf.number_cwt_peaks(x, int(constants.COMMON_FREQ/25))], f_names=["abs_energy", "kurtosis", "entropy", "num_peaks"], vectorize="full")[:]
        train_edss = datasets.Flattener(datasets.EdfFFTDatasetTransformer(list(zip(train_edss[0], train_edss[1])), freq_bins=freq_bins, is_pandas_data=False), n_process=12)[:]
        pkl.dump(train_edss, open("extracted_val_features_v2.pkl", "wb"))
extract_val_features()
def extract_test_features():
    with h5py.File("/datadrive/TUH_EEG/MUPS/data/cross_sub_TUH_EEG_denoised/cross_subject_data_test.hdf5") as trainValFile:
        train_edss = (trainValFile['test_x'][()], trainValFile['test_y'][()])
        trainCoherData = datasets.CoherenceTransformer(list(zip(train_edss[0], train_edss[1])), is_pandas=False)
        trainSHED = datasets.SimpleHandEngineeredDataset(list(zip(train_edss[0], train_edss[1])), n_process=12, is_pandas_data=False, features=[tsf.abs_energy, tsf.kurtosis, tsf.sample_entropy, lambda x: tsf.number_cwt_peaks(x, int(constants.COMMON_FREQ/25))], f_names=["abs_energy", "kurtosis", "entropy", "num_peaks"], vectorize="full")[:]
        train_edss = datasets.Flattener(datasets.EdfFFTDatasetTransformer(list(zip(train_edss[0], train_edss[1])), freq_bins=freq_bins, is_pandas_data=False), n_process=12)[:]
        pkl.dump(train_edss, open("extracted_test_features_v2.pkl", "wb"))
extract_test_features()

starting simple handEngineeredData


/home/mohammed/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:8: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  


Starting 12 processes
retrieving: 0
retrieving: 250
retrieving: 500
retrieving: 750
retrieving: 1000
retrieving: 1250
retrieving: 1500
retrieving: 1750
retrieving: 2000
retrieving: 2250
retrieving: 2500
retrieving: 2750
retrieving: 3000
retrieving: 3250
retrieving: 3500
retrieving: 3750
retrieving: 4000
retrieving: 4250
retrieving: 4500
retrieving: 4750
retrieving: 5000
retrieving: 5250
retrieving: 5500
retrieving: 5750
retrieving: 6000
retrieving: 6250
retrieving: 6500
retrieving: 6750
retrieving: 7000
retrieving: 7250
retrieving: 7500
retrieving: 7750
retrieving: 8000
retrieving: 8250
retrieving: 8500
retrieving: 8750
retrieving: 9000
retrieving: 9250
retrieving: 9500
retrieving: 9750
retrieving: 10000
retrieving: 10250
retrieving: 10500
retrieving: 10750
retrieving: 11000
retrieving: 11250
retrieving: 11500
retrieving: 11750
retrieving: 12000
retrieving: 12250
retrieving: 12500
retrieving: 12750
retrieving: 13000
retrieving: 13250
retrieving: 13500
retrieving: 13750
retrieving: 1400

/home/mohammed/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:16: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  app.launch_new_instance()


Starting 12 processes
retrieving: 0
retrieving: 250
retrieving: 500
retrieving: 750
retrieving: 1000
retrieving: 1250
retrieving: 1500
retrieving: 1750
retrieving: 2000
retrieving: 2250
retrieving: 2500
retrieving: 2750
retrieving: 3000
retrieving: 3250
retrieving: 3500
retrieving: 3750
retrieving: 4000
retrieving: 4250
retrieving: 4500
retrieving: 4750
retrieving: 5000
retrieving: 5250
retrieving: 5500
retrieving: 5750
retrieving: 6000
retrieving: 6250
retrieving: 6500
retrieving: 6750
retrieving: 7000
retrieving: 7250
retrieving: 7500
retrieving: 7750
retrieving: 8000
retrieving: 8250
retrieving: 8500
retrieving: 8750
retrieving: 9000
retrieving: 9250
retrieving: 9500
retrieving: 9750
retrieving: 10000
retrieving: 10250
retrieving: 10500
retrieving: 10750
retrieving: 11000
retrieving: 11250
retrieving: 11500
retrieving: 11750
retrieving: 12000
retrieving: 12250
retrieving: 12500
retrieving: 12750
retrieving: 13000
retrieving: 13250
retrieving: 13500
retrieving: 13750
retrieving: 1400

/home/mohammed/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:24: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.


Starting 12 processes
retrieving: 0
retrieving: 250
retrieving: 500
retrieving: 750
retrieving: 1000
retrieving: 1250
retrieving: 1500
retrieving: 1750
retrieving: 2000
retrieving: 2250
retrieving: 2500
retrieving: 2750
retrieving: 3000
retrieving: 3250
retrieving: 3500
retrieving: 3750
retrieving: 4000
retrieving: 4250
retrieving: 4500
retrieving: 4750
retrieving: 5000
retrieving: 5250
retrieving: 5500
retrieving: 5750
retrieving: 6000
retrieving: 6250
retrieving: 6500
retrieving: 6750
retrieving: 7000
retrieving: 7250
retrieving: 7500
retrieving: 7750
retrieving: 8000
retrieving: 8250
retrieving: 8500
retrieving: 8750
retrieving: 9000
retrieving: 9250
retrieving: 9500
retrieving: 9750
retrieving: 10000
retrieving: 10250
retrieving: 10500
retrieving: 10750
retrieving: 11000
retrieving: 11250
retrieving: 11500
retrieving: 11750
retrieving: 12000
retrieving: 12250
retrieving: 12500
retrieving: 12750
retrieving: 13000
retrieving: 13250
retrieving: 13500
retrieving: 13750
retrieving: 1400

In [12]:

trainCoherData = np.stack([datum.values for datum in [datum[0] for datum in wfdata.CoherenceTransformer(simple_edss(train_edss), columns_to_use=constants.SYMMETRIC_COLUMN_SUBSET, n_process=n_process, is_pandas=False)[:]]])
validCoherData = np.stack([datum.values for datum in [datum[0] for datum in wfdata.CoherenceTransformer(simple_edss(valid_edss), columns_to_use=constants.SYMMETRIC_COLUMN_SUBSET, n_process=n_process, is_pandas=False)[:]]])
testCoherData = np.stack([datum.values for datum in  [datum[0] for datum in wfdata.CoherenceTransformer(simple_edss(test_edss), columns_to_use=constants.SYMMETRIC_COLUMN_SUBSET, n_process=n_process, is_pandas=False)[:]]])
trainSHED = wfdata.SimpleHandEngineeredDataset(simple_edss(train_edss), n_process=n_process, is_pandas_data=False, features=[tsf.abs_energy, tsf.sample_entropy, lambda x: tsf.number_cwt_peaks(x, int(constants.COMMON_FREQ/25))], f_names=["abs_energy", "entropy", "num_peaks"], vectorize="full")[:]
validSHED = wfdata.SimpleHandEngineeredDataset(simple_edss(valid_edss), n_process=n_process, is_pandas_data=False, features=[tsf.abs_energy, tsf.sample_entropy, lambda x: tsf.number_cwt_peaks(x, int(constants.COMMON_FREQ/25))], f_names=["abs_energy", "entropy", "num_peaks"], vectorize="full")[:]
testSHED = wfdata.SimpleHandEngineeredDataset(simple_edss(test_edss), n_process=n_process, is_pandas_data=False, features=[tsf.abs_energy, tsf.sample_entropy, lambda x: tsf.number_cwt_peaks(x, int(constants.COMMON_FREQ/25))], f_names=["abs_energy", "entropy", "num_peaks"], vectorize="full")[:]
train_edss = read.Flattener(read.EdfFFTDatasetTransformer(train_edss, freq_bins=freq_bins, is_pandas_data=False), n_process=n_process)[:]
valid_edss = read.Flattener(read.EdfFFTDatasetTransformer(valid_edss, freq_bins=freq_bins, is_pandas_data=False), n_process=n_process)[:]
test_edss = read.Flattener(read.EdfFFTDatasetTransformer(test_edss, freq_bins=freq_bins, is_pandas_data=False), n_process=n_process)[:]

NameError: name 'np' is not defined